This is a python script which is a replicated MapBasic code from a file named **BusLines_Frequency_F6.MBX** 


_Important Note: Python v3.9.1 is needed since it comes compiled with SQLite version 3.33.0. This version of SQLite has introduced some new SQL JOIN statements which are used in the script._


**Create folder structure:**

```
D:\Routelines
        \data
        \db
        \rlc
        \shp

\data -> put here "NOCTable.csv" and CIF files in subfolders by region
\db   -> here will be created sqlite db file
\rlc  -> here will be the files related to "Route Line Creator" (input CSV and output result)
\shp  -> here will be created final shapefile
```

In [1]:
import os
import sys
import csv
import math
import mmap
import sqlite3

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

print('Python v' + sys.version)
print('------------------------------------------------------------------------------')
print('Pandas v' + pd.__version__)
print('NumPy v' + np.__version__)
print('SQLite v' + sqlite3.sqlite_version)

Python v3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
------------------------------------------------------------------------------
Pandas v1.2.4
NumPy v1.19.4
SQLite v3.34.1


#### 1.  CIF data extraction functions

In [2]:
def mapcount(filename):
    print("Counting number of lines...")
    f = open(filename, "r+")
    buf = mmap.mmap(f.fileno(), 0)
    lines = 0
    readline = buf.readline
    while readline():
        lines += 1
    print(f'{lines} lines in the file!')
    return lines

In [3]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [4]:
def insert_with_progress(df, tablename):
    chunksize = int(len(df) / 10) # 10%
    with tqdm(total=len(df)) as pbar:
        for i, cdf in enumerate(chunker(df, chunksize)):
            cdf.to_sql(tablename, conn, schema='rl', if_exists='append', index=False)
            pbar.update(chunksize)
            pbar.set_description(f'Insert "{region}" region to SQLite')
    rows_inserted = c.execute('''SELECT * FROM '''+ tablename + ''' PTStops;''')
    print(tablename + " rows number: ", len(c.fetchall()))

In [5]:
# Reading CIF file

def read_cif(cif_file):
    print(f'CIF file size is {round(os.stat(cif_file).st_size / (1024 * 1024), 2)} MB')   
    count_lines = mapcount(cif_file)
    chunk_size=1000000
    chunks = []
    loops = math.ceil(count_lines/chunk_size)
    i=0
    with tqdm(total = loops, file = sys.stdout) as pbar:
        reader = pd.read_csv(cif_file, names=['CODE'], header=None, sep='!', iterator=True)
        while i <= loops:
            try:
                i+=1
                chunk = reader.get_chunk(chunk_size)
                chunks.append(chunk)
                pbar.set_description('Loading raw CIF data')
                pbar.update(1)
            except StopIteration:
                loop = False
                cif_data = pd.concat(chunks, ignore_index=True)
                pbar.update(1)
    pbar.set_description('Done!')
    pbar.close()
    return cif_data


In [6]:
# PTStops table creation

def GetPtStops(cif_data, region, tablename):
    print("Extracting QB nodes in progress...")
    QB = cif_data['CODE'].str.extract('(^QB.*)').dropna()
    QB.columns = ['CODE']
    print("QB nodes extracted:", len(QB))
    COLUMN_NAMES = ['NaptanID','Xcoord','Ycoord']
    PTStops = pd.DataFrame(columns=COLUMN_NAMES)
    PTStops['NaptanID'] = QB['CODE'].str.slice(start=3, stop=15).str.rstrip()
    PTStops['Xcoord'] = QB['CODE'].str.slice(start=15, stop=21)
    PTStops['Ycoord'] = QB['CODE'].str.slice(start=23, stop=29)
    insert_with_progress(PTStops, tablename)
    return PTStops    

In [7]:
# Routes exctraction

def GetRouteDataToSQLite(cif_df):
    
    # temp list for RouteLines1 columns
    l_BMRouteID_1 = []
    l_BM_StartStopID_1 = []
    l_AnodeStopID = []
    l_BnodeStopID = []

    # temp list for MainTable columns
    l_OperatorCode = []
    l_ServiceNum = []
    l_BM_RouteID_2 = []
    l_BM_StartStopID_2 = []
    l_Direction = []
    l_StopID = []
    l_DeptTime = []
    l_Seq = []
    l_Mon = []
    l_Tue = []
    l_Wed = []
    l_Thu = []
    l_Fri = []
    l_Sat = []
    l_Sun = []

    # RouteLines1 to sqlite3
    c.execute('''DROP TABLE IF EXISTS RouteLines1;''')
    conn.commit()  
    c.execute('''CREATE TABLE RouteLines1(
                                          Region TEXT,
                                          BMRouteID TEXT,
                                          BM_StartStopID TEXT,
                                          AnodeStopID TEXT,
                                          BnodeStopID TEXT,
                                          AnodeXCoord INTEGER,
                                          AnodeYCoord INTEGER,
                                          BnodeXCoord INTEGER,
                                          BnodeYCoord INTEGER,
                                          id INTEGER PRIMARY KEY
                                          )
    ''')

    # MainTable to sqlite3
    c.execute('''DROP TABLE IF EXISTS MainTable;''')
    conn.commit()      
    c.execute('''CREATE TABLE MainTable(
                                        OperatorCode TEXT,
                                        ServiceNum TEXT,
                                        BM_RouteID TEXT,
                                        BM_StartStopID TEXT,
                                        Direction TEXT,
                                        StopID TEXT,
                                        DeptTime INTEGER,
                                        Seq INTEGER,
                                        Mon Integer,
                                        Tue Integer,
                                        Wed Integer,
                                        Thur Integer,
                                        Fri Integer,
                                        Sat Integer,
                                        Sun Integer,
                                        TotalWeekly Integer,
                                        id INTEGER PRIMARY KEY
                                        )
                                        ''')
    conn.commit()
    
#   -----------
    print("Data extraction from CIF to lists...")
    
    BankHolOnly = "" 

    for x in cif_df['CODE']:
        if x[0:2]=='QS':
    #         -----------------------
    #         QS
    #         -----------------------
            SeqNo = 0
            AnodeStr = "XXXX"
            RouteIDFull = x[38:65]
    #         print("1-",RouteIDFull)
            RouteIDV = RouteIDFull[:4].rstrip()
    #         print("2-",RouteIDV)
            RouteIDFull = x[37:65]
    #         print("3-",RouteIDFull)        
            BankHolOnly = RouteIDFull[0]
    #         print("4-",BankHolOnly)
            if BankHolOnly != "":
                pass
            RouteIDFull =x[3:]
    #         print("5-"+RouteIDFull)                
            RouteOpV = RouteIDFull[:4].rstrip()
    #         print("6-"+RouteOpV)
            ServiceIDv = RouteIDV
            RouteIDFull = x[64:]
    #         print("7-"+RouteIDFull)        
            DirectionV = RouteIDFull[0]
            RouteIDV = region + "_" + RouteOpV + "_" +  RouteIDV + "_" + DirectionV
    #         print("8-"+RouteIDV)
    #         RouteIDFull = x[29:]
            MonV = int(x[29:][0])
    #         print("MonV =", MonV)
            TueV = int(x[30:][0])
    #         print("TueV =", TueV)
            WedV = int(x[31:][0])
    #         print("WedV =", WedV)
            ThuV = int(x[32:][0])
    #         print("ThuV =", ThuV)
            FriV = int(x[33:][0])
    #         print("FriV =", FriV)
            SatV = int(x[34:][0])
    #         print("SatV =", SatV)
            SunV = int(x[35:][0])
    #         print("SunV =", SunV)
    #         print("------------")
        if BankHolOnly == "B":
            BankLoop = BankLoop + 1
            if BankLoop == 1:
                print("WARNING: Bank holiday services exist!")
        else:
            if x[0:2] in ('QO','QI','QT'):
    #             print(x[0:2])
    #             -----------------------
    #             QO / QI / QT
    #             -----------------------
                RouteIDFull = x[2:]
    #             print("9 -", RouteIDFull)
                RSStopID = RouteIDFull[:12].rstrip()
    #             print("10-", RSStopID)
                StopArea = RouteIDFull[:4].rstrip()
                BnodeStr = RSStopID
    #             print("StopArea -",StopArea)
    #             print("BnodeStr -",BnodeStr)
                RouteIDFull = x[14:]
    #             print("11-", RouteIDFull)
                TimeNum = RouteIDFull[:4]
    #             print("12-", TimeNum)
                SeqNo = SeqNo + 1
                if SeqNo == 1:
                    StartStopID = RSStopID
                if SeqNo > 1:
    #                 print("RouteLines1:")
                    l_BMRouteID_1.append(RouteIDV)
                    l_BM_StartStopID_1.append(StartStopID)
                    l_AnodeStopID.append(AnodeStr)
                    l_BnodeStopID.append(BnodeStr)
                AnodeStr = RSStopID
                if int(TimeNum) >= 0 and int(TimeNum) < 2400:
    #                 print("MainTable:")
                    l_OperatorCode.append(RouteOpV)
                    l_ServiceNum.append(ServiceIDv)
                    l_BM_RouteID_2.append(RouteIDV)
                    l_BM_StartStopID_2.append(StartStopID)
                    l_Direction.append(DirectionV)
                    l_StopID.append(RSStopID)
                    l_DeptTime.append(TimeNum)
                    l_Seq.append(SeqNo)
                    l_Mon.append(MonV)
                    l_Tue.append(TueV)
                    l_Wed.append(WedV)
                    l_Thu.append(ThuV)
                    l_Fri.append(FriV)
                    l_Sat.append(SatV)
                    l_Sun.append(SunV)
    

    # Pandas dataframe creations

    # - RouteLines1
    print("Loading RouteLines1 to Pandas dataframe...")
    
    RouteLines1 = pd.DataFrame(
        {'BMRouteID':l_BMRouteID_1,
         'BM_StartStopID':l_BM_StartStopID_1,
         'AnodeStopID':l_AnodeStopID,
         'BnodeStopID':l_BnodeStopID
        })
    
    # - MainTable
    print("Loading MainTable to Pandas dataframe...")
    
    MainTable = pd.DataFrame(
        {'OperatorCode':l_OperatorCode,
         'ServiceNum':l_ServiceNum,
         'BM_RouteID':l_BM_RouteID_2,
         'BM_StartStopID':l_BM_StartStopID_2,
         'Direction':l_Direction,
         'StopID':l_StopID,
         'DeptTime':l_DeptTime,
         'Seq':l_Seq,
         'Mon':l_Mon,
         'Tue':l_Tue,
         'Wed':l_Wed,
         'Thur':l_Thu,
         'Fri':l_Fri,
         'Sat':l_Sat,
         'Sun':l_Sun
        })
    
    # ------------------------------------------------------------------------------------------------------------
    # RouteLines1.info()
    # RouteLines1.head()
    # MainTable.info()
    # MainTable.head()
    
    # ------------------------------------------------------------------------------------------------------------
    #  Upload to db

    print("Uploading RouteLines1 df to SQLite...")    
    RouteLines1.to_sql('RouteLines1', conn, schema='rl' ,if_exists='append', index = False) # , chunksize = 10000

    print("Uploading MainTable df to SQLite...")    
    MainTable.to_sql('MainTable', conn, schema='rl' ,if_exists='append', index = False)     # , chunksize = 10000
    
    # ------------------------------------------------------------------------------------------------------------
    # PTStops_temp
    
    print("Updating 'PTStops_temp'...")
    
#     c.execute('''DROP TABLE IF EXISTS PTStops_temp;''')
#     c.execute('''CREATE TABLE PTStops_temp 
#                  AS SELECT NaptanID, Xcoord, Ycoord 
#                  FROM PTStops 
#                  GROUP BY NaptanID, Xcoord, Ycoord;
#     ''')
    c.execute('''INSERT INTO PTStops_temp (NaptanID, Xcoord, Ycoord)
                 SELECT NaptanID, Xcoord, Ycoord FROM PTStops GROUP BY NaptanID, Xcoord, Ycoord;
    ''')    
    conn.commit()
    
    # ------------------------------------------------------------------------------------------------------------
    # UPDATE RouteLines1...
    
    print("Updating RouteLines1 RouteID ...")
    
    c.execute('''UPDATE RouteLines1
                 SET BMRouteID = TempUpdRouteID.BMRouteID || '_' || CAST(ROUND(TempUpdRouteID.Xcoord/1000.0,0) AS INTEGER) || '_' || CAST(ROUND(TempUpdRouteID.Ycoord/1000.0,0) AS INTEGER) 
                 FROM (SELECT BMRouteID, Xcoord, Ycoord, id FROM RouteLines1, PTStops_temp WHERE RouteLines1.BM_StartStopID = PTStops_temp.NaptanID) AS TempUpdRouteID
                 WHERE TempUpdRouteID.id = RouteLines1.id;
    ''')
    conn.commit()

    # ------------------------------------------------------------------------------------------------------------
    # UPDATE Maintable...
    
    print("Updating Maintable RouteID...")
    
    c.execute('''UPDATE Maintable
                 SET BM_RouteID = TempUpdRouteID.BM_RouteID || '_' || CAST(ROUND(TempUpdRouteID.Xcoord/1000.0,0) AS INTEGER) || '_' || CAST(ROUND(TempUpdRouteID.Ycoord/1000.0,0) AS INTEGER) 
                 FROM (SELECT * FROM Maintable, PTStops_temp WHERE Maintable.BM_StartStopID = PTStops_temp.NaptanID) AS TempUpdRouteID
                 WHERE TempUpdRouteID.id = Maintable.id;
    ''')
    conn.commit()

#### 2. CreateFreq

In [8]:
def TempGetFreq(ColName, depttime1, depttime2):
    c.execute('''CREATE TABLE TempGetFreq AS SELECT * FROM Maintable WHERE ''' + ColName + ''' = 1 AND DeptTime >= ''' + str(depttime1) + ''' AND DeptTime < ''' + str(depttime2) + ''' AND Seq = 1;''')
    conn.commit()
    c.execute('''CREATE TABLE SumFreq AS SELECT BM_RouteID, SUM(Seq) AS SumFreq FROM TempGetFreq GROUP BY BM_RouteID;''')
    conn.commit()    

def UpdFreqVal(UpdCol, round_value):
    c.execute('''UPDATE RLFreq SET ''' + UpdCol + ''' = UpdFreqVal.SumFreq/''' + str(round_value) + '''
                 FROM (SELECT RLFreq.BM_RouteID, OperatorCode, ServiceNum, Direction, OperatorName,
                              MonEarly, MonAM, MonBP, MonEP, MonOP, MonNight,
                              TueEarly, TueAM, TueBP, TueEP, TueOP, TueNight, 
                              WedEarly, WedAM, WedBP, WedEP, WedOP, WedNight,
                              ThurEarly, ThurAM, ThurBP, ThurEP, ThurOP, ThurNight,
                              FriEarly, FriAM, FriBP, FriEP, FriOP, FriNight,
                              SatEarly, SatAM, SatBP, SatEP, SatOP, SatNight, 
                              SunEarly, SunAM, SunBP, SunEP, SunOP, SunNight, 
                              id, SumFreq 
                       FROM RLFreq, SumFreq 
                       WHERE RLFReq.BM_RouteID = SumFreq.BM_RouteID) AS UpdFreqVal 
                 WHERE RLFreq.id = UpdFreqVal.id;''')
    conn.commit()
    c.execute('''UPDATE RLFreq SET ''' + UpdCol + ''' = 0 WHERE ''' + UpdCol + ''' is NULL;''')
    conn.commit()
    c.execute('''UPDATE RLFreq SET ''' + UpdCol + ''' = ROUND(''' + UpdCol + ''', 2);''')
    conn.commit()
    c.execute('''DROP TABLE IF EXISTS TempGetFreq;''')
    conn.commit()
    c.execute('''DROP TABLE IF EXISTS SumFreq;''')
    conn.commit()
    

def CreateFreq():
    
    ColNames = {1:'Mon', 2:'Tue', 3:'Wed', 4:'Thur', 5:'Fri', 6:'Sat', 7:'Sun'}
    
    DepTimes = {
                'AM':[2.0, 700,900],
                'BP':[7.0, 900,1600],
                'EP':[2.0, 1600,1800],
                'OP':[6.0, 1800,2400],
                'Night':[3.0, 0,300],
                'Early':[3.0, 400,700]
               }    
    c.execute('''DROP TABLE IF EXISTS RLFreq;''')    
    conn.commit()    
    c.execute('''CREATE TABLE RLFreq(
       BM_RouteID    TEXT,
       OperatorCode  TEXT,
       ServiceNum    TEXT,
       Direction     TEXT,
       OperatorName  TEXT,
       MonEarly      REAL,
       MonAM         REAL,
       MonBP         REAL,
       MonEP         REAL,
       MonOP         REAL,
       MonNight      REAL,
       TueEarly      REAL,
       TueAM         REAL,
       TueBP         REAL,
       TueEP         REAL,
       TueOP         REAL,
       TueNight      REAL,
       WedEarly      REAL,
       WedAM         REAL,
       WedBP         REAL,
       WedEP         REAL,
       WedOP         REAL,
       WedNight      REAL,
       ThurEarly     REAL,
       ThurAM        REAL,
       ThurBP        REAL,
       ThurEP        REAL,
       ThurOP        REAL,
       ThurNight     REAL,
       FriEarly      REAL,
       FriAM         REAL,
       FriBP         REAL,
       FriEP         REAL,
       FriOP         REAL,
       FriNight      REAL,
       SatEarly      REAL,
       SatAM         REAL,
       SatBP         REAL,
       SatEP         REAL,
       SatOP         REAL,
       SatNight      REAL,
       SunEarly      REAL,
       SunAM         REAL,
       SunBP         REAL,
       SunEP         REAL,
       SunOP         REAL,
       SunNight      REAL,
       id            INTEGER PRIMARY KEY
     );
    ''')
    
    c.execute('''INSERT INTO RLFreq (
                                     BM_RouteID, 
                                     OperatorCode, 
                                     ServiceNum, 
                                     Direction
                                     )
                 SELECT BM_RouteID, 
                        OperatorCode,
                        ServiceNum,
                        Direction
                 FROM MainTable GROUP BY BM_RouteID ORDER BY BM_RouteID;
    ''')
    conn.commit()

    c.execute('''UPDATE RLFreq SET OperatorName = OperatorPublicName 
                 FROM (SELECT * From RLFreq, NOCTable 
                       WHERE OperatorCode = NOCCODE 
                       AND NOT OperatorCode = '') AS UpdOP
                 WHERE UpdOP.id = RLFreq.id;''')
    conn.commit()
    
    for key1, value1 in ColNames.items():
        print("Freq. Calculations for " + value1 + "...")
        for key2, value2 in DepTimes.items():
#             print(key1, "ColName=" + str(value1+key2), value2[0],value2[1],value2[2])
#             print("TempGetFreq", str(value1), value2[1], value2[2])
            TempGetFreq(str(value1), value2[1], value2[2])
#             print("UpdFreqVal", str(value1+key2), value2[0])
            UpdFreqVal(str(value1+key2), value2[0])


#### 3. ABSorter

In [9]:
def ABsorter():
    
    print("Creation of ABNodes and updating...")
    
    c.execute('''DROP TABLE IF EXISTS ABNodes;''')
    conn.commit()  
    c.execute('''CREATE TABLE ABNodes AS Select AnodeStopID, BnodeStopID, AnodeXcoord, AnodeYcoord, BnodeXcoord, BnodeYcoord, id From RouteLines1 group by AnodeStopID, BnodeStopID;''')
    conn.commit()  

    c.execute('''UPDATE ABNodes
    SET AnodeXcoord = UpdAnode.Xcoord, AnodeYcoord = UpdAnode.Ycoord 
    FROM (Select * from ABNodes, PTStops_temp Where AnodeStopid = NaptanID) AS UpdAnode
    WHERE UpdAnode.id = ABNodes.id;''')
    conn.commit()  

    c.execute('''UPDATE ABNodes
    SET BnodeXcoord = UpdBnode.Xcoord, BnodeYcoord = UpdBnode.Ycoord 
    FROM (Select * from ABNodes, PTStops_temp Where BnodeStopid = NaptanID) AS UpdBnode
    WHERE UpdBnode.id = ABNodes.id;''')
    conn.commit()  

#### 4. ABNodesUK export

In [10]:
def export_csv():
    print("Export ABNodesUK to CSV for 'RouteLineCreator.exe'")
    db_df = pd.read_sql_query("SELECT AnodeStopID, BnodeStopID, ifnull(AnodeXCoord, 0) AS AnodeXCoord, ifnull(AnodeYCoord, 0) AS AnodeYCoord, ifnull(BnodeXCoord, 0) AS BnodeXCoord,ifnull(BnodeYCoord, 0) AS BnodeYCoord FROM ABNodesUK", conn)
    db_df.to_csv(rlc_input_path + 'ABNodesUK.csv', index=False, quoting=csv.QUOTE_NONNUMERIC, na_rep=0) 
    print("ABNodesUK.csv created!")


## Main routine

##### Please set the project folder path here:

In [11]:
project = "D:/Routelines/"

data folders:

In [12]:
cif_path = project + 'data/'
db_path = project + 'db/'
rlc_input_path = project + 'rlc/'

CIF files by regions:

In [13]:
RegionID = ['EA'] #,'W','NE','EM','WM','SW','Y','NW','S','SE','L']
cif ='/ATCO_BUS.cif'
db_name = "routelines.db"
tablename ="PTStops"

CIF file as a single file:

In [14]:
# region = 'single CIF'
# cif_s ='Bus_1.cif'
# # db_name_s = 'QB_nodes.db'
# db_name_s = db_name_r
# tablename_s ="PTStops_s"

Create SQLite database 'routelines.db':

In [15]:
# set the enivronment variable to enable "ExportSHP" function

os.environ['SPATIALITE_SECURITY'] = 'relaxed'

In [16]:
conn = sqlite3.connect(db_path + db_name)
conn.enable_load_extension(True)
conn.execute('SELECT load_extension("mod_spatialite")')   
conn.execute('SELECT InitSpatialMetaData(1);')
c = conn.cursor()

Import table 'NOCTable':

In [17]:
NOCTable = pd.read_csv(cif_path + 'NOCTable.csv')
NOCTable.to_sql('NOCTable', conn, schema='rl', if_exists='replace', index = False, chunksize = 100000)

Create  table 'PtStops_temp':

In [18]:
c.execute('''CREATE TABLE PTStops_temp(NaptanID TEXT, Xcoord TEXT, Ycoord TEXT);''')
conn.commit()

Data extraction from CIF files:

In [19]:
region_counter = 0

for region in RegionID:
    print("========================================================")
    cif_file = cif_path + region + cif
    print(cif_file)    
    region_counter += 1
    cif_df = read_cif(cif_file)
    PTStops = GetPtStops(cif_df, region, tablename)
    GetRouteDataToSQLite(cif_df)
    CreateFreq()
    ABsorter()
    if region_counter == 1:
        print("Creation of 'ABNodesUK', 'RLFreqUK', 'RoutelinesUK'...")
        c.execute('''CREATE TABLE ABNodesUK AS SELECT * FROM ABNodes;''')
        conn.commit()        
        c.execute('''CREATE TABLE RLFreqUK AS SELECT * FROM RLFreq;''')
        conn.commit()
        c.execute('''CREATE TABLE RoutelinesUK AS Select Region, BMRouteID, AnodeStopID, BnodeStopID From RouteLines1 Group By BMRouteID, AnodeStopID, BnodeStopID;''')
        conn.commit()  
    elif region_counter > 1:
        print("Updating 'ABNodesUK', 'RLFreqUK', 'RoutelinesUK'...")
        c.execute('''UPDATE ABNodes SET AnodeXcoord = 999999 FROM (Select * from ABNodes, ABNodesUK 
                     WHERE ABNodes.AnodeStopID = ABNodesUK.AnodeStopID AND ABNodes.BnodeStopID = ABNodesUK.BnodeStopID) 
                     AS DelAB WHERE DelAB.id = ABNodes.id;''')
        conn.commit()
        c.execute('''DROP TABLE IF EXISTS QualAB;''')
        conn.commit()
        c.execute('''CREATE TABLE QualAB AS Select * FROM ABNodes WHERE AnodeXcoord < 999999  OR AnodeXcoord IS NULL;''')
        conn.commit()  
        c.execute('''INSERT INTO ABNodesUK (AnodeStopID, BnodeStopID, AnodeXCoord, AnodeYCoord,BnodeXCoord,BnodeYCoord, id) SELECT * FROM QualAB;''')
        conn.commit()  
        c.execute('''INSERT INTO RLFreqUK SELECT * FROM RLFreq;''')
        conn.commit()
        c.execute('''DROP TABLE IF EXISTS RouteLinesGrp;''')  
        conn.commit()
        c.execute('''CREATE TABLE RouteLinesGrp AS SELECT Region, BMRouteID, AnodeStopID, BnodeStopID FROM RouteLines1 GROUP BY Region, BMRouteID, AnodeStopID, BnodeStopID;''')
        conn.commit()  
        c.execute('''INSERT INTO RouteLinesUK SELECT * FROM RouteLinesGrp;''')
        conn.commit()        
    else:
        print("No processed regions")


D:/Routelines/data/EA/ATCO_BUS.cif
CIF file size is 13.08 MB
Counting number of lines...
390975 lines in the file!


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting QB nodes in progress...
QB nodes extracted: 8306


  0%|          | 0/8306 [00:00<?, ?it/s]

PTStops rows number:  8306
Data extraction from CIF to lists...
Loading RouteLines1 to Pandas dataframe...
Loading MainTable to Pandas dataframe...
Uploading RouteLines1 df to SQLite...
Uploading MainTable df to SQLite...
Updating 'PTStops_temp'...
Updating RouteLines1 RouteID ...
Updating Maintable RouteID...
Freq. Calculations for Mon...
Freq. Calculations for Tue...
Freq. Calculations for Wed...
Freq. Calculations for Thur...
Freq. Calculations for Fri...
Freq. Calculations for Sat...
Freq. Calculations for Sun...
Creation of ABNodes and updating...
Creation of 'ABNodesUK', 'RLFreqUK', 'RoutelinesUK'...


#### ABNodesUK.csv export

In [20]:
export_csv()

Export ABNodesUK to CSV for 'RouteLineCreator.exe'
ABNodesUK.csv created!


In [21]:
conn.close()
print("Connection to routelines.db closed!")

Connection to routelines.db closed!


<font color='red' style="font-size:30px"><b>Done!</b></font> 

In [22]:

# MapBasic EA region calculations     : 03m:24sec
# Python+SQLite EA region calculations: 00m:10sec  ~20x faster

# MapBasic All regions calculations     : 4h:45m:00sec
# Python+SQLite All regions calculations: 0h:17m:10sec  ~17x faster